In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
import random
from time import sleep
from pandas import json_normalize
import pandas as pd
import pickle
import ast
from sklearn.preprocessing import StandardScaler

#### Import 100 hot songs

In [2]:
hot100songs = pd.read_csv('100songs.csv')

#### Import clustered song collection

In [3]:
clustered_songs = pd.read_csv('clustered_songs.csv')

#### Load kmeans model

In [4]:
kmeans = pickle.load(open('kmeans_model.sav', 'rb'))

#### Load Scaler

In [5]:
scaler = pickle.load(open('standard_scaler.sav', 'rb'))

#### Connection to Spotify API

In [6]:
secrets_file = open("C:/Users/lisab/OneDrive/Desktop/DA_labs/Week_6/Spotify_API.txt")

In [7]:
string = secrets_file.read()

In [8]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [9]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['Client ID'],
                                                           client_secret=secrets_dict['Client Secret']))

In [22]:
def getArtists(artist_list):
    #convert string containing list to list
    artist_list = ast.literal_eval(artist_list)
    if len(artist_list) > 1:
        artists = '0'
        for artist in artist_list:
            artists = artists+', '+artist
            artists = artists.replace('0', '')
    else:
        artists = artist_list[0]
    return artists

#### SongRecommender Function

In [32]:
def checkSongAndRecommend():
    song_input = input("Enter your favorite song: ")
    artist_input = input("Enter the artist: ")
    
    #lowercase song input and hot100songs title column
    song_input_lower = song_input.lower()
    hot100songs['title_lower'] = hot100songs['title'].str.lower()
    
    #Checking if song title is in top100
    if len(hot100songs[hot100songs.title_lower.str.contains(song_input_lower)]) > 0:
        random_song = hot100songs['title'][random.randint(0, len(hot100songs)-1)]
        random_artist = hot100songs.loc[hot100songs.title == random_song, 'artists'].values[0]
        
        return 'Look at this new hot song: '+random_song+' from '+random_artist
    
    #If song is not in top100 list:
    else:
        #Search uri for input song on spotify
        #song_spotify = sp.search(q=song_input, limit=1)
        song_spotify = sp.search(q=song_input+' '+artist_input, limit=1)
        song_uri = song_spotify['tracks']['items'][0]['uri']
        
        #Collect audio features for input song
        song_audio_features = sp.audio_features(song_uri)
        audio_feat_df = pd.DataFrame(song_audio_features)
        audio_feat_df_corr = audio_feat_df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                          'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']].copy()
        
        #Scale audio feature columns
        audio_feat_df_corr_prep = scaler.transform(audio_feat_df_corr)
        
        # Predicting / assigning the clusters:
        clusters = kmeans.predict(audio_feat_df_corr_prep)
        
        #take clustered_songs dataframe and search for a song & artist with the same number as clusters result
        clustered_songs_input_cluster = clustered_songs[clustered_songs['cluster'] == clusters[0]].reset_index()
        random_song_output = clustered_songs_input_cluster['title'][random.randint(0, len(clustered_songs_input_cluster['title'])-1)]
        random_artist_output = clustered_songs_input_cluster.loc[clustered_songs_input_cluster.title == random_song_output, 
                                'artists'].values[0]
        random_artist_output = getArtists(random_artist_output)
        
        return 'Found a similar song: '+random_song_output+' from '+random_artist_output

#### Test the complete function

In [33]:
checkSongAndRecommend()

Enter your favorite song: Party in the USA
Enter the artist: Miley Cyrus


'Found a similar song: Break the Ice from Britney Spears'